Corriger une prédiction HTR
===
<br>
D'après la démarche expliquée dans la documentation du projet DAHN, on procède selon les étapes suivantes.

On privilégie le format XML-Page pour exporter et réimporter les prédictions HTR dans le logiciel de transcription.

In [1]:
from py.constantes import XMLaCORRIGER, XMLCORRIGEES
print(f"""On place les prédictions HTR dans le dossier : {XMLaCORRIGER}""")

On place les prédictions HTR dans le dossier : ./xmlPage-aCorriger/


Exécuter le script **spellcheck_texts_PAGEXML.py**, qui analyse chaque mot des prédictions HTR et génère pour chaque fichier XML à corriger un dictionnaire au format Json contenant des propositions de corrections : 

In [3]:
!python3 py/spellcheck_texts_PAGEXML.py

Le fichier ./xmlPage-aCorriger/CdS02_Konv002-02_0175.xml est en cours de lecture.
^C
Traceback (most recent call last):
  File "/home/sbiay/dhi/venv/lib/python3.8/site-packages/spellchecker/spellchecker.py", line 251, in _check_if_should_check
    float(word)
ValueError: could not convert string to float: '(siné会̀'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "py/spellcheck_texts_PAGEXML.py", line 138, in <module>
    spellcheck_texts_page_XML()
  File "py/spellcheck_texts_PAGEXML.py", line 122, in spellcheck_texts_page_XML
    'lem': spell.correction(mot),
  File "/home/sbiay/dhi/venv/lib/python3.8/site-packages/spellchecker/spellchecker.py", line 143, in correction
    return max(self.candidates(word), key=self.word_probability)
  File "/home/sbiay/dhi/venv/lib/python3.8/site-packages/spellchecker/spellchecker.py", line 162, in candidates
    tmp = self.known([x for x in self.__edit_distance_alt(res)])
  File "/home/s

Corriger à la main les entrées du dictionnaire de chacun des fichiers générés dans le dossier [dictPages/](py/dictPages/).

**Attention** : il faut bien veiller à…
- Ne pas transformer de mots qui seraient justes dans un autre contexte (comme le mot **"tout"** dans l'exemple suivant, qui ne doit pas être corrigé en "août" bien que le contexte s'y prête, au risque de modifier des "tout" pertinents dans leur propre contexte)

- Éliminer les propositions de lemmes erronées, en modifiant le lemme comme "null" (**"Paris"** dans l'exemple suivant)

Il n'est pas nécessaire de transformer les mots justes dont le lemme est indiqué comme "null", car ils seront intégrés au réservoir des mots connus dès leurs prochaine rencontre (ici l'exemple de **"homme"**).

```json
{
    "tout": {
      "lem": null,
      "ctxt": "Dyck, ce 3 TOUT 1818"
   },
    "Paris": {
      "lem": "pris",
      "ctxt": "c est qu il retournera à PARIS dans le mois"
   },
    "homme": {
      "lem": null,
      "ctxt": "que ce brave HOMME se rétablisse"
   },
}
```

Une fois cette tâche accomplie, on applique le script [dictCDSmanip.py](./py/dictCDSmanip.py). Voyons dans un premier temps son rôle et les options qu'il propose :

In [6]:
!python3 py/dictCDSmanip.py --help

Usage: dictCDSmanip.py [OPTIONS]

  Ce script prend comme paramètres une source consistant en un fichier Json
  d'entrée contenu dans le dossier dictPages le compare au contenu de
  dictCDS.json, intègre les entrées nouvelles et retourne des messages
  d'alertes pour les entrées générant un conflit d'intégration. :param
  fichierdictpages: fichier Json issu de la correction automatisée d'une
  page de transcription :type fichierdictpages: Json :returns: None

Options:
  -f, --file TEXT  Nom de fichier contenu dans le dossier ./py/dictPages/
  -A, --all        Prend tous les fichiers du dossier ./py/dictPages/
  --help           Show this message and exit.


In [26]:
# Ajouter l'option qui convient et relancer le script
!python3 py/dictCDSmanip.py -A

Traitement du fichier DictCdS02_Konv002-02_0190.json
Traitement du fichier DictCdS02_Konv002-02_0191.json
Traitement du fichier DictCdS02_Konv002-02_0175.json
Traitement du fichier DictCdS02_Konv002-02_0067.json
Traitement du fichier DictCdS02_Konv002-02_0073.json
Le dictionnaire dictCDS.json est désormais à jour.


Appliquer le dictionnaire de correction aux fichiers XML grâce au script [text_correction_XML.py](./py/text_correction_XML.py) :

In [30]:
!python3 py/text_correction_XML.py

Le fichier CdS02_Konv002-02_0175.xml a été corrigé avec succès
Le fichier CdS02_Konv002-02_0073.xml a été corrigé avec succès
Le fichier CdS02_Konv002-02_0191.xml a été corrigé avec succès
Le fichier CdS02_Konv002-02_0190.xml a été corrigé avec succès
Le dictionnaire dictCDS.json est désormais à jour.


On finalise le processus de correction avec les étapes suivantes :
1. Réimporter les fichiers corrigés dans eScriptorium ;
2. Finaliser manuellement la correction des transcriptions ;
3. Télécharger la verité de terrain pour la sauvegarder au format XML-Alto.

In [9]:
print(f"""On place les vérités de terrain dans le dossier : {XMLCORRIGEES}""")

On place les vérités de terrain dans le dossier : ./xmlPage-corrigees/
